In [ ]:
import os
from typing import Literal
from functools import partial


import numpy as np
import torch
from torch.utils.data import DataLoader as TorchDataloader
from torch_geometric.loader import DataLoader as GraphDataloader
from torch_scatter import scatter_max
from tqdm import tqdm


from geometric_governance.util import (
    Logger,
    RangeOrValue,
    get_value,
    get_max,
    OUTPUT_DIR,
)
from geometric_governance.data import (
    SetDataset,
    generate_synthetic_election,
    get_scoring_function_winners,
)
from geometric_governance.model import (
    MessagePassingElectionModel,
    DeepSetElectionModel,
)

device = torch.device(0) if torch.cuda.is_available() else torch.device("cpu")

config = {
    "num_voters_range": (3, 50),
    "num_candidates_range": (2, 10),
    "train_dataset_size": 100_000,
    # "train_dataset_size": 500_000,
    "train_batch_size": 128,
    "train_num_epochs": 200,
    "checkpoint_interval": 5,
    "val_num_voters": 75,
    "val_num_candidates": 15,
    "test_num_voters": 100,
    "test_num_candidates": 20,
    # "train_iterations_per_epoch": 200,
    "train_iterations_per_epoch": 100,
    "eval_dataset_size": 1_000,
    "learning_rate": 0.001,
    # "learning_rate": 0.01,
    "clip_grad_norm": 1.0,
    "use_monotonicity_loss": False,
    "monotonicity_loss_batch_size": 32,
    "voting_rule": "copeland",
    "representation": "graph",
}


def generate_rule_dataset(
    dataset_size: int,
    num_voters_range: RangeOrValue,
    num_candidates_range: RangeOrValue,
    dataloader_batch_size: int,
    top_k_candidates: int | None,
    voting_rule: Literal["plurality", "borda", "copeland"],
    representation: Literal["set", "graph"],
    seed: int,
    recompute: bool = True,
):
    rng = np.random.default_rng(seed=seed)
    dataset_file = os.path.join(
        OUTPUT_DIR,
        f"rule_dataset_{dataset_size}_{num_voters_range}_{num_candidates_range}_{representation}_{voting_rule}_{seed}.pt",
    )
    if os.path.exists(dataset_file) and not recompute:
        with open(dataset_file, "rb") as f:
            dataset = torch.load(f, weights_only=False)
    else:
        dataset = []

        generated_count = 0
        with tqdm(range(dataset_size)) as pbar:
            while generated_count < dataset_size:
                num_voters = get_value(num_voters_range, rng)
                num_candidates = get_value(num_candidates_range, rng)

                election_data = generate_synthetic_election(
                    num_voters=num_voters, num_candidates=num_candidates, rng=rng
                )

                match voting_rule:
                    case "plurality":
                        scores = election_data.positional_ballots[0]
                        winners = get_scoring_function_winners(scores)
                    case "borda":
                        scoring = torch.tensor(
                            list(range(num_candidates, 0, -1)), dtype=torch.float32
                        )
                        scores = scoring @ election_data.positional_ballots
                        winners = get_scoring_function_winners(scores)
                    case "copeland":
                        scores = election_data.tournament_embedding.sum(dim=1)
                        winners = get_scoring_function_winners(scores)
                    case _:
                        raise ValueError("Unknown voting rule.")

                if winners.max() < 1.0:
                    # Tie
                    continue

                if representation == "graph":
                    graph = election_data.to_bipartite_graph(
                        top_k_candidates, vote_data="ranking"
                    )
                    graph.y = election_data.voter_utilities.sum(dim=0)
                    graph.winners = winners
                    dataset.append(graph)
                elif representation == "set":
                    pad_shape = get_max(num_candidates_range) - num_candidates
                    voter_preferences = election_data.voter_preferences_alt
                    voter_preferences = torch.nn.functional.pad(
                        voter_preferences, (0, pad_shape, 0, 0)
                    )
                    winners = torch.nn.functional.pad(winners, (0, pad_shape))
                    dataset.append((voter_preferences, winners))

                generated_count += 1
                pbar.update(1)

        with open(dataset_file, "wb") as f:
            torch.save(dataset, f)

    if representation == "graph":
        dataloader = GraphDataloader(
            dataset, batch_size=dataloader_batch_size, shuffle=True
        )
    elif representation == "set":
        voter_preferences_list = [x[0] for x in dataset]
        winner_list = [x[1] for x in dataset]
        set_dataset = SetDataset(voter_preferences_list, winner_list)
        dataloader = TorchDataloader(
            set_dataset,
            batch_size=dataloader_batch_size,
            shuffle=True,
            collate_fn=SetDataset.collate_fn,
        )
    return dataloader


generate_dataset = partial(
    generate_rule_dataset,
    voting_rule=config["voting_rule"],
    representation=config["representation"],
    dataloader_batch_size=config["train_batch_size"],
    top_k_candidates=None,
    recompute=False,
)

train_dataloader = generate_dataset(
    dataset_size=config["train_dataset_size"],
    num_voters_range=config["num_voters_range"],
    num_candidates_range=config["num_candidates_range"],
    seed=42,
)

val_dataloader = generate_dataset(
    dataset_size=config["eval_dataset_size"],
    num_voters_range=config["val_num_voters"],
    num_candidates_range=config["val_num_candidates"],
    seed=16180,
)

if config["representation"] == "graph":
    test_num_candidates = config["test_num_candidates"]
elif config["representation"] == "set":
    test_num_candidates = get_max(config["num_candidates_range"])

test_dataloader = generate_dataset(
    dataset_size=config["eval_dataset_size"],
    num_voters_range=config["test_num_voters"],
    num_candidates_range=test_num_candidates,
    seed=314159,
)

config["train_iterations_per_epoch"] = min(
    config["train_iterations_per_epoch"], len(iter(train_dataloader))
)

In [ ]:
if config["representation"] == "graph":
    election_model = MessagePassingElectionModel(
        node_emb_dim=256, edge_emb_dim=64, num_layers=4, edge_dim=1
    )
elif config["representation"] == "set":
    election_model = DeepSetElectionModel(
        get_max(config["num_candidates_range"]), embedding_size=155
    )
parameter_count = sum(p.numel() for p in election_model.parameters() if p.requires_grad)
print(f"parameter_count: {parameter_count}")


election_model.to(device=device)
optim = torch.optim.Adam(election_model.parameters(), lr=config["learning_rate"])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optim, T_max=config["train_num_epochs"], eta_min=1e-4
)
experiment_name = f"{config['representation']}-election-{config['voting_rule']}"
with (
    Logger(
        experiment_name=experiment_name,
        config=config,
        mode="online",
    ) as logger,
    tqdm(range(config["train_num_epochs"])) as pbar,
):
    best_validation_accuracy: float = 0.0

    for epoch in range(config["train_num_epochs"]):
        # Train
        train_loss = 0
        train_rule_loss = 0
        train_monotonicity_loss = 0
        train_welfare = 0
        total, correct = 0, 0

        election_model.train()

        train_iter = iter(train_dataloader)

        for _ in range(config["train_iterations_per_epoch"]):
            optim.zero_grad()

            # Rule Loss
            if config["representation"] == "graph":
                data = next(train_iter).to(device=device)
                data.edge_attr.requires_grad = True
                out = election_model(data)
                winners = data.winners
                rule_loss = -(out * winners).sum() / config["train_batch_size"]
            elif config["representation"] == "set":
                X, index, y = next(train_iter)
                X = X.to(device=device)
                y = y.to(device=device)
                index = index.to(device=device)
                out = election_model(X, index=index)
                winners = y
                rule_loss = torch.nn.functional.cross_entropy(out, winners)

            # Monotonicity loss
            monotonicity_loss = 0

            if config["use_monotonicity_loss"] and config["representation"] == "graph":
                candidates = data.candidate_idxs.nonzero()
                perm = torch.randperm(candidates.size(0))[
                    : config["monotonicity_loss_batch_size"]
                ]
                for i in perm:
                    candidate_idx = candidates[i]
                    edge_idxs = data.edge_index[1] == candidate_idx
                    grad = torch.autograd.grad(
                        outputs=out[i], inputs=data.edge_attr, create_graph=True
                    )[0]
                    monotonicity_loss += torch.where(
                        grad[edge_idxs] < 0,
                        -grad[edge_idxs],
                        torch.zeros_like(grad[edge_idxs]),
                    ).mean()
            monotonicity_loss /= config["monotonicity_loss_batch_size"]

            loss = rule_loss + monotonicity_loss
            loss.backward()

            torch.nn.utils.clip_grad_norm_(
                election_model.parameters(), config["clip_grad_norm"]
            )
            optim.step()
            scheduler.step()

            if config["representation"] == "graph":
                batch_idxs = data.batch[data.candidate_idxs]
                _, predicted = scatter_max(out, batch_idxs)
                _, predicted_ground = scatter_max(winners, batch_idxs)
            elif config["representation"] == "set":
                _, predicted = torch.max(out, dim=1)
                _, predicted_ground = torch.max(winners, dim=1)

            total += predicted_ground.shape[0]
            correct += (predicted == predicted_ground).sum().item()
            if config["representation"] == "graph":
                welfare = data.y[predicted].mean()
                train_welfare += welfare.item()
            train_loss += loss.item()
            train_rule_loss += rule_loss.item()
            if config["use_monotonicity_loss"]:
                train_monotonicity_loss += monotonicity_loss.item()

        train_loss /= config["train_iterations_per_epoch"]
        train_rule_loss /= config["train_iterations_per_epoch"]
        train_monotonicity_loss /= config["train_iterations_per_epoch"]
        train_welfare /= config["train_iterations_per_epoch"]
        train_accuracy = correct / total

        if epoch % config["checkpoint_interval"] == 0:
            torch.save(
                election_model, os.path.join(logger.checkpoint_dir, f"model_{epoch}.pt")
            )

        logger.log(
            {
                "train/total_loss": train_loss,
                "train/rule_loss": train_rule_loss,
                "train/monotonicity_loss": train_monotonicity_loss,
                "train/welfare": train_welfare,
                "train/accuracy": train_accuracy,
            }
        )

        # Validation
        val_loss = 0
        val_accuracy = 0
        if config["representation"] == "graph":
            val_welfare = 0
            election_model.eval()
            total, correct = 0, 0
            with torch.no_grad():
                for data_ in val_dataloader:
                    data = data_.to(device=device)
                    out = election_model(data)
                    rule_loss = -(out * data.winners).sum() / config["train_batch_size"]
                    if config["representation"] == "graph":
                        batch_idxs = data.batch[data.candidate_idxs]
                        _, predicted = scatter_max(out, batch_idxs)
                        _, predicted_ground = scatter_max(data.winners, batch_idxs)
                    elif config["representation"] == "set":
                        _, predicted = torch.max(out, dim=1)
                        _, predicted_ground = torch.max(data.winners, dim=1)

                    total += predicted_ground.shape[0]
                    correct += (predicted == predicted_ground).sum().item()

                    welfare = data.y[predicted].mean()
                    val_loss += rule_loss.item()
                    val_welfare += welfare.item()

            val_loss /= len(val_dataloader)
            val_welfare /= len(val_dataloader)
            val_accuracy = correct / total

            if val_accuracy > best_validation_accuracy:
                print(f"New best accuracy: {val_accuracy}")
                torch.save(
                    election_model,
                    os.path.join(logger.checkpoint_dir, "model_best.pt"),
                )
                best_validation_accuracy = val_accuracy

            logger.log(
                {
                    "val/rule_loss": val_loss,
                    "val/accuracy": val_accuracy,
                    "val/welfare": val_welfare,
                }
            )
        logger.commit()

        pbar.set_postfix(
            {
                "train_rule_loss": train_rule_loss,
                "train_accuracy": train_accuracy,
                "val_rule_loss": val_loss,
                "val_accuracy": val_accuracy,
            }
        )
        pbar.update(1)

In [ ]:
if config["representation"] == "graph":
    election_model = torch.load(
        os.path.join(logger.checkpoint_dir, "model_best.pt"), weights_only=False
    )
    election_model.eval()

    test_loss = 0
    total, correct = 0, 0
    with torch.no_grad():
        for data_ in test_dataloader:
            data = data_.to(device=device)
            out = election_model(data)
            rule_loss = -(out * data.winners).sum() / config["train_batch_size"]
            batch_idxs = data.batch[data.candidate_idxs]
            _, predicted = scatter_max(out, batch_idxs)
            _, predicted_ground = scatter_max(data.winners, batch_idxs)
            total += predicted_ground.shape[0]
            correct += (predicted == predicted_ground).sum().item()
            test_loss += rule_loss.item()

    test_loss /= len(test_dataloader)
    test_accuracy = correct / total

    print(f"Test | Accuracy {test_accuracy} | Loss {test_loss}")
elif config["representation"] == "set":
    election_model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for X, index, y in test_dataloader:
            X = X.to(device)
            index = index.to(device)
            y = y.to(device)
            p_y = election_model(X, index)
            _, predicted = torch.max(p_y, dim=1)
            _, predicted_ground = torch.max(y, dim=1)
            total += y.shape[0]
            correct += (predicted == predicted_ground).sum().item()

    accuracy = correct / total
    print(f"Accuracy: {accuracy}")